# Natural Language Processing - Text Preprocessing

## Libraries and settings

In [1]:
# Libraries
import os
import re
import string
import numpy as np
import pandas as pd
from pprint import pprint

import nltk

# Import only once
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.chunk import tree2conlltags
from nltk.chunk import conlltags2tree
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Current working directory
print('Current working directory:', os.getcwd())

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data] Downloading package omw-1.4 to /home/vscode/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vscode/nltk_data...


Current working directory: /workspaces/data_analytics/Week_11


[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Defining documents

In [16]:
# Defining documents (=sentenses)
d1 = 'He eats a sandwich for lunch'
d2 = 'She ate a curry for dinner.'
d3 = 'The lunch was eaten cold'

corpus_01 = d1 + ' ' + d2 + ' ' + d3
corpus_01

'He eats a sandwich for lunch She ate a curry for dinner. The lunch was eaten cold'

## Text preprocessing
#### Steps:
- Text to lowercase
- Removing punctuations
- Tokenization
- Removal of stop words
- Lemmatization

### Text to lowercase

In [17]:
# Text to lowercase function
def text_lowercase(text):
    return text.lower()

# Text to lowercase
corpus_02 = text_lowercase(corpus_01)
corpus_02

'he eats a sandwich for lunch she ate a curry for dinner. the lunch was eaten cold'

### Removing punctuation

In [18]:
# Remove punctuation function
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Remove punctuation
corpus_03 = remove_punctuation(corpus_02)
corpus_03

'he eats a sandwich for lunch she ate a curry for dinner the lunch was eaten cold'

### Tokenize text & removal of stopwords

In [19]:
# Show english stopwords
eng_stopwords = set(stopwords.words('english'))
print("List of english stopwords:")
print(eng_stopwords)

List of english stopwords:
{'ma', 'to', 'as', 'does', 'up', 'ain', 'be', 'having', 'just', 'yours', 'are', 'but', 'won', 'by', 'wasn', 'how', 'isn', 'been', 'at', 'before', "wouldn't", 'not', 'no', 'shan', 'hadn', 'now', 'only', 'me', 'yourselves', 'his', 'here', 'have', 'down', 'them', 'while', 'himself', 'haven', 'or', 'both', 'shouldn', 'i', "you've", 'him', 'myself', 'those', 'ourselves', "doesn't", 'between', 'other', 'these', 'll', 'has', "wasn't", 'you', 'most', 'hasn', 'there', 'when', 'ours', 'after', 'y', 'can', "it's", 'don', 'each', 'needn', 'mightn', 'and', "won't", 'about', 'the', "should've", 'o', "aren't", 'of', 'why', 'whom', 'had', 'itself', 't', "you'd", 'her', 'couldn', 'they', 'we', 'wouldn', 'on', 'such', 'for', "isn't", 'a', 'again', 'mustn', 'under', 'off', 'very', 've', 'should', 'below', 're', "haven't", "you're", 'more', 'didn', "that'll", 'were', 'out', 'she', 'same', "shan't", 'will', 'above', 'through', 'd', 'hers', 'until', "hadn't", 'did', 'being', 'duri

In [20]:
# Function for tokenization and the removal of stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text
 
# Remove stopwords
corpus_04 = remove_stopwords(corpus_03)
print(corpus_04, end="")

['eats', 'sandwich', 'lunch', 'ate', 'curry', 'dinner', 'lunch', 'eaten', 'cold']

### Lemmatization

In [21]:
# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize string function
def lemmatize_word(text):
    word_tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    return lemmas

# Lemmatize
lem = []
for i in corpus_04:
    lem.append(lemmatize_word(i))

# Nested list to list
corpus_05 = [' '.join([str(x) for x in lst]) for lst in lem]

print('Before lemmatization:')
print(corpus_04, '\n')

print('After lemmatization:')
print(corpus_05, end="")

Before lemmatization:
['eats', 'sandwich', 'lunch', 'ate', 'curry', 'dinner', 'lunch', 'eaten', 'cold'] 

After lemmatization:
['eat', 'sandwich', 'lunch', 'eat', 'curry', 'dinner', 'lunch', 'eat', 'cold']

## Redefine the text corpus (pre-processed)

In [22]:
# We will use the lemmatized words above to re-define our corpus 
corpus = ['eat sandwich lunch', 
          'eat curry dinner', 
          'lunch eat cold']

## Document-term matrix with ngram_range=(1,1)

In [23]:
# Vectorizer with ngram_range=(1,1)
vectorizer = CountVectorizer(min_df=0.0, ngram_range=(1,1))

# Transform 
count = vectorizer.fit_transform(corpus)
 
# Create dataframe
df_count = pd.DataFrame(count.toarray(),
                        columns=vectorizer.get_feature_names_out())

print('Document-term matrix')
print(df_count)

Document-term matrix
   cold  curry  dinner  eat  lunch  sandwich
0     0      0       0    1      1         1
1     0      1       1    1      0         0
2     1      0       0    1      1         0


## Document-term matrix with ngram_range=(2,2)

In [24]:
# Vectorizer with with ngram_range=(2,2)
vectorizer = CountVectorizer(min_df=0.0, ngram_range=(2,2))

# Transform 
count = vectorizer.fit_transform(corpus)
 
# Create dataframe
df_count = pd.DataFrame(count.toarray(),
                        columns=vectorizer.get_feature_names_out())

print('Document-term matrix')
print(df_count)

Document-term matrix
   curry dinner  eat cold  eat curry  eat sandwich  lunch eat  sandwich lunch
0             0         0          0             1          0               1
1             1         0          1             0          0               0
2             0         1          0             0          1               0


## Term frequency-inverse document frequency (TF-IDF)
- For details see: https://www.learndatasci.com/glossary/tf-idf-term-frequency-inverse-document-frequency

### Term Frequency (TF)

In [25]:
# Compute Term Frequency (TF)
words_set = set()
for doc in corpus:
    words = doc.split(' ')
    words_set = words_set.union(set(words))
    
print('Number of words in the corpus:',len(words_set), '\n')
print('The words in the corpus: \n', words_set)

# Number of documents in the corpus
n_docs = len(corpus)

# Number of unique words in the corpus 
n_words_set = len(words_set)

df_tf = pd.DataFrame(np.zeros((n_docs, n_words_set)), 
                     columns=list(words_set))

print("\nTerm Frequency (TF):")
for i in range(n_docs):
    # Words in the document
    words = corpus[i].split(' ')
    for w in words:
        df_tf[w][i] = df_tf[w][i] + (1 / len(words))
        
print(df_tf.round(4))

Number of words in the corpus: 6 

The words in the corpus: 
 {'curry', 'cold', 'dinner', 'lunch', 'eat', 'sandwich'}

Term Frequency (TF):
    curry    cold  dinner   lunch     eat  sandwich
0  0.0000  0.0000  0.0000  0.3333  0.3333    0.3333
1  0.3333  0.0000  0.3333  0.0000  0.3333    0.0000
2  0.0000  0.3333  0.0000  0.3333  0.3333    0.0000


### Inverse Document Frequency (IDF)

In [26]:
# Computing Inverse Document Frequency (IDF)
print("\nInverse Document Frequency (IDF):")

idf = {}

for w in words_set:
    
    # k = number of documents that contain this word
    k = 0
    
    for i in range(n_docs):
        if w in corpus[i].split():
            k += 1
            
    idf[w] =  np.log10(n_docs / k).round(4)
    
    print(f'{w:>15}: {idf[w]:>10}')


Inverse Document Frequency (IDF):
          curry:     0.4771
           cold:     0.4771
         dinner:     0.4771
          lunch:     0.1761
            eat:        0.0
       sandwich:     0.4771


### Term Frequency - Inverse Document Frequency (TF-IDF)

In [27]:
# Computing TF-IDF
df_tf_idf = df_tf.copy()

for w in words_set:
    for i in range(n_docs):
        df_tf_idf[w][i] = df_tf[w][i] * idf[w]

print('\nTF-IDF:')
print(df_tf_idf.round(4))


TF-IDF:
   curry   cold  dinner   lunch  eat  sandwich
0  0.000  0.000   0.000  0.0587  0.0     0.159
1  0.159  0.000   0.159  0.0000  0.0     0.000
2  0.000  0.159   0.000  0.0587  0.0     0.000


## Part-of-Speach (POS) tagging
For meaning of POS-tags see: https://pythonexamples.org/nltk-pos-tagging

In [29]:
text = '''Arca is a Venezuelan musician and record producer based in Barcelona, Spain. She[a] initially began releasing music under the name of Nuuro.'''

def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

sent = preprocess(text)
pattern = 'NP: {<DT>?<JJ>*<NN>}'

cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)

iob_tagged = tree2conlltags(cs)

# Print the POS-tags
pprint(iob_tagged)

[('Arca', 'NNP', 'O'),
 ('is', 'VBZ', 'O'),
 ('a', 'DT', 'O'),
 ('Venezuelan', 'JJ', 'O'),
 ('musician', 'JJ', 'O'),
 ('and', 'CC', 'O'),
 ('record', 'NN', 'B-NP'),
 ('producer', 'NN', 'B-NP'),
 ('based', 'VBN', 'O'),
 ('in', 'IN', 'O'),
 ('Barcelona', 'NNP', 'O'),
 (',', ',', 'O'),
 ('Spain', 'NNP', 'O'),
 ('.', '.', 'O'),
 ('She', 'PRP', 'O'),
 ('[', 'VBD', 'O'),
 ('a', 'DT', 'O'),
 (']', 'NNP', 'O'),
 ('initially', 'RB', 'O'),
 ('began', 'VBD', 'O'),
 ('releasing', 'VBG', 'O'),
 ('music', 'NN', 'B-NP'),
 ('under', 'IN', 'O'),
 ('the', 'DT', 'B-NP'),
 ('name', 'NN', 'I-NP'),
 ('of', 'IN', 'O'),
 ('Nuuro', 'NNP', 'O'),
 ('.', '.', 'O')]


Interpretation:

NNP: Proper Noun, Singular
VBZ: Verb, 3rd person sing. present takes
DT: Determiner
JJ: Adjective
CC: Coordinating Conjunction


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [15]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.2.0-1016-azure
Datetime: 2023-11-27 11:57:33
Python Version: 3.10.13
-----------------------------------
